In [ ]:
import ipyleaflet
import ogr, os
from IPython.display import Image
from shapely.geometry import mapping, shape
from shapely.geometry import box as shBox
import pandas as pd
import geopandas as gpd
import gdal
from shapely.geometry import Polygon
from shapely.geometry import mapping, LineString, MultiLineString
import shapefile
#Needed external file: download US coast outline from: https://geodata.lib.berkeley.edu/catalog/stanford-xv279yj9196
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)


#Footprint data
foot= '.../'
#ROI name
roi="NorthCarolina"
#Set buffer distance in decimal degreees
dist=0.01


watercolor = basemap_to_tiles(basemaps.Esri.WorldImagery)

m = Map(layers=(watercolor, ), center=(50, 354), zoom=5)


dc = DrawControl(marker={'shapeOptions': {'color': '#0000FF'}},
                 rectangle={'shapeOptions': {'color': '#0000FF'}},
                 circle={'shapeOptions': {'color': '#0000FF'}},
                 circlemarker={},
                 )
def handle_draw(target, action, geo_json):
    print(action)
    print(geo_json)

dc.on_draw(handle_draw)
m.add_control(dc)


m

In [ ]:
bounds = GeoJSON(data=dc.last_draw)


#Import Line file of US Coast:
US_cst= gpd.read_file('.../usa_coastl.shp')#Downloaded file from https://geodata.lib.berkeley.edu/catalog/stanford-xv279yj9196
US_cst=US_cst.to_crs('EPSG:4326')



bnd=gpd.read_file(bounds)
bnd=bnd.set_crs('EPSG:4326')

#clip coastal polyline
roi_coast=geopandas.clip(US_cst,bnd)
roi_coast=roi_coast.to_crs('EPSG:4326')
buffer=roi_coast.buffer(dist)

print('buffer created')

json=gpd.GeoDataFrame(gpd.read_file(foot))
json_clipped = geopandas.clip(json, buffer)
json_clipped.to_file('.../ROI_Bldgs.geojson', driver='GeoJSON')